In [ ]:
import imports
import drill

In [ ]:
def encode_cyclical(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [ ]:
def create_plot(data):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['timestamp'], 
                             y=data['co2_ppm'], 
                             mode='markers',
                             marker=dict(
                                 color=(
                                    (data['presence'] == 0)).astype('int'),
                                    colorscale=[[0, 'yellow'], [1, 'blue']])))
    return fig

In [ ]:
#df = get_PIR_data()
#df.to_json('data.json')

In [ ]:
df = pd.read_json('data.json') 

In [ ]:
plt.scatter(df, x='timestamp', y='co2_ppm', color='presence')

In [ ]:
# timestamp etwas leichter zu verarbeiten, wenn als Integer gespeichert
df_test = df

# Fehlerhafte Tage zwischen 18.10. und 03.11. entfernen
#df_test= df.drop(df[(df['timestamp'] > pd.to_datetime(1634518800, unit='s', origin='unix')) & 
#                (df['timestamp'] < pd.to_datetime(1635901200, unit='s', origin='unix'))].index)

# Zusätzlich zwischen 08.11. und 22.11.
#df_test= df.drop(df[(df['timestamp'] > pd.to_datetime(1636376400, unit='s', origin='unix')) & 
#                (df['timestamp'] < pd.to_datetime(1637586000, unit='s', origin='unix'))].index)

# alle zwischen 18.10. und 22.11. entfernen
df_test = df.drop(df[(df_test['timestamp'] > pd.to_datetime(1634518800, unit='s', origin='unix')) & 
                    (df['timestamp'] < pd.to_datetime(1637586000, unit='s', origin='unix'))].index)


df_test = df_test.assign(hoursMinutesSeconds=lambda d: (d['timestamp'].dt.hour.astype('int') * 10000 + 
                                                        d['timestamp'].dt.minute.astype('int') * 100 + 
                                                       d['timestamp'].dt.second.astype('int')))

df_test['hour_sin'] = np.sin(2 * np.pi * df_test['hoursMinutesSeconds']/235959.0)
df_test['hour_cos'] = np.cos(2 * np.pi * df_test['hoursMinutesSeconds']/235959.0)

In [ ]:
plt.scatter(df_test, x='timestamp', y='co2_ppm', color='presence')

In [ ]:
# erhoeht sich die Genauigkeit, wenn Datenset ausgeglichener ist?
# Idee bei ca. 8 Stunden Anwesenheit:
# Entferne 8 Nachtstunden von 22:00-05:00, 
# sodass von 24 stunden 8 Anwesenheits-Stunden und 8 Abwesenheits-Stunden uebrig bleiben
df_test.drop(df_test[(df_test['hoursMinutesSeconds'] < 50000) |
                     (df_test['hoursMinutesSeconds'] > 220000)].index, inplace=True)

In [ ]:
# Template für mehrere Plots in einem Graphen
#fig = create_plot(df_test)
#df_test['co2_ppmShifted'] = df_test['co2_ppm'].shift(12)
#fig.add_trace(go.Scatter(x=df_test['timestamp'], 
#                             y=df_test['co2_ppmShifted'], 
#                             mode='lines',
#                             line=dict(width=2),
#                             marker=dict(
#                                 color=(
#                                    (df_test['presence'] == 0)).astype('int'),
#                                    colorscale=[[0, 'green'], [1, 'red']])))

#fig.show()
#plt.scatter(df_test, x='timestamp', y='co2_ppm', color='presence')

In [ ]:
# Steigungen mit Abstaenden 5, 30 und 60 Minuten einfuegen
df_test['co2_ppm_deltaOne'] = df_test['co2_ppm'] - df_test.shift(1)['co2_ppm']
df_test['co2_ppm_deltaSix'] = df_test['co2_ppm'] - df_test.shift(6)['co2_ppm']
df_test['co2_ppm_deltaTwelve'] = df_test['co2_ppm'] - df_test.shift(12)['co2_ppm']
#df_test['co2_ppm_deltaMinusOne'] = df_test.shift(-1)['co2_ppm'] - df_test['co2_ppm']
#df_test['co2_ppm_deltaMinusSix'] = df_test.shift(-6)['co2_ppm'] - df_test['co2_ppm']
#df_test['co2_ppm_deltaMinusTwelve'] = df_test.shift(-12)['co2_ppm'] - df_test['co2_ppm']

In [ ]:
# Wochentag einfuegen
# verringert Genauigkeit, weil wahrscheinlich zu "verlaesslich"

#df_test['dayOfWeek'] = df['timestamp'].dt.dayofweek
#df_test = df_test.drop(df_test[df_test.dayOfWeek > 4].index)

In [ ]:
# Ausreisser mit Interquartile Range (IQR) und Tukey's Method loeschen
# Idee: Fenster wird an fast allen Tagen ab ca. 1200-1300 ppm geoeffnet
# -> was veraendert sich, wenn die Werte darüber entfernt werden und das Datenset so "geglaettet" wird?
# Erhoeht Genauigkeit um ca. 2%
x = df_test['co2_ppm']
q1 = np.percentile(x, 12)
q3 = np.percentile(x, 88)
iqr = q3 - q1
floor = q1 - 1.5*iqr
ceiling = q3 + 1.5*iqr
outlier_indices = list(x.index[(x < floor) | (x > ceiling)])
outlier_values = list(x[outlier_indices])
print(outlier_values)

In [ ]:
df_test = df_test.drop(index=outlier_indices)

In [ ]:
# timestamp, presence, temperatur und humidity entfernen
# temp/humid erhoehen Genauigkeit deutlich, da relativ unverlaesslich -> von zu vielen aeusseren Faktoren abhaengig
df_test = df_test.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_timestamp = df_test['timestamp']
y_presence = df_test['presence']
X_presence = df_test.drop(['timestamp', 'hoursMinutesSeconds', 'presence', 'temperature_celsius', 'relative_humidity_percent'], axis=1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X_presence, y_presence, test_size=0.2, random_state=1, shuffle=False)
Xtrain

In [ ]:
# shift des trainings-sets um 5 minuten in die Vergangenheit
# -> Test ob Model auch in die Zukunft Erwartungen treffen kann
#ytrain = ytrain.shift(-6)
#ytrain.dropna(axis=0, how='any', inplace=False)
#ytrain = ytrain.replace(np.nan, 0)

In [ ]:
# Random Forest Classification
model = RandomForestClassifier(n_estimators=250)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)
#print(metrics.classification_report(ypred, ytest))
accuracy_score(ytest, ypred)

In [ ]:
# plot feature-importance
feature_imp = pd.Series(model.feature_importances_,index=Xtrain.columns).sort_values(ascending=False)
# Creating a bar plot
ax = sns.barplot(x=feature_imp, y=feature_imp.index)
ax.set(xlabel='Importance', ylabel='Feature')

In [ ]:
# Random Forest Regression
model = RandomForestRegressor(n_estimators=250)
model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest)

In [ ]:
df_valid = Xtest.copy()
df_valid['timestamp'] = df_timestamp
df_valid['prediction'] = ypred
df_valid['co2_ppm'] = df_test['co2_ppm']
df_valid['presence'] = ytest

plt.scatter(df_valid, x='timestamp', y='co2_ppm', color='prediction')